In [56]:
import project1 as p1
import utils
import numpy as np
import random
from string import punctuation, digits

In [2]:
def get_order(n_samples):
    try:
        with open(str(n_samples) + '.txt') as fp:
            line = fp.readline()
            return list(map(int, line.split(',')))
    except FileNotFoundError:
        random.seed(1)
        indices = list(range(n_samples))
        random.shuffle(indices)
        return indices

In [49]:
def accuracy(preds, targets):
    """
    Given length-N vectors containing predicted and target labels,
    returns the percentage and number of correct predictions.
    """
    return (preds == targets).mean()

In [4]:
def hinge_loss_single(feature_vector, label, theta, theta_0):
    """
    Finds the hinge loss on a single data point given specific classification
    parameters.

    Args:
        feature_vector - A numpy array describing the given data point.
        label - A real valued number, the correct classification of the data
            point.
        theta - A numpy array describing the linear classifier.
        theta_0 - A real valued number representing the offset parameter.


    Returns: A real number representing the hinge loss associated with the
    given data point and parameters.
    """
    
    hinge_loss = max(0, 1 - label * (np.dot(feature_vector, theta) + theta_0))

    return hinge_loss

    raise NotImplementedError


In [5]:
feature_vector = np.array([5.85757831, 9.48411088, 4.71348922, 2.43062214, 4.27445804, 8.6418371, 1.03627538, 1.49498883, 1.42915177, 4.3454958, ])
label = 1.
theta = np.array([3.89173371, 3.7477549, 9.95500721, 9.86402351, 1.73976234, 3.71439254, 3.27476477, 2.76343474, 2.38647062, 6.69378231])
theta_0 = 1.

In [6]:
def hinge_loss_full(feature_matrix, labels, theta, theta_0):
    """
    Finds the total hinge loss on a set of data given specific classification
    parameters.

    Args:
        feature_matrix - A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        theta - A numpy array describing the linear classifier.
        theta_0 - A real valued number representing the offset parameter.


    Returns: A real number representing the hinge loss associated with the
    given dataset and parameters. This number should be the average hinge
    loss across all of the points in the feature matrix.
    """
    hinge_loss = []

    for i in range(len(labels)):
        hinge_loss.append(max(0, 1 - labels[i] * (np.dot(feature_matrix[i, :], theta) + theta_0)))

    hinge_loss = np.array(hinge_loss)
    
    return np.mean(hinge_loss)

In [7]:
feature_matrix = np.array([[1, 0], [1,0]])
labels = np.array([1., -1.])
theta = np.array([1, 0])
theta_0 = 1.

In [8]:
def perceptron_single_step_update(
        feature_vector,
        label,
        current_theta,
        current_theta_0):
    """
    Properly updates the classification parameter, theta and theta_0, on a
    single step of the perceptron algorithm.

    Args:
        feature_vector - A numpy array describing a single data point.
        label - The correct classification of the feature vector.
        current_theta - The current theta being used by the perceptron
            algorithm before this update.
        current_theta_0 - The current theta_0 being used by the perceptron
            algorithm before this update.

    Returns: A tuple where the first element is a numpy array with the value of
    theta after the current update has completed and the second element is a
    real valued number with the value of theta_0 after the current updated has
    completed.
    """
    def less_than_zero(x):
        neg_epsilon = -0.01
        pos_epsilon = 0.01
        return x < pos_epsilon
    
    mitad = np.dot(feature_vector, current_theta) + current_theta_0

    if less_than_zero(label * mitad):
        current_theta += label * feature_vector
        current_theta_0 += label

    return (current_theta, current_theta_0)

In [9]:
feature_vector = np.array([1., 0.])
label = 1.
current_theta = np.array([1., 1.])
current_theta_0 = 0.

In [10]:
def perceptron(feature_matrix, labels, T):
    """
    Runs the full perceptron algorithm on a given set of data. Runs T
    iterations through the data set, there is no need to worry about
    stopping early.

    NOTE: Please use the previously implemented functions when applicable.
    Do not copy paste code from previous parts.

    NOTE: Iterate the data matrix by the orders returned by get_order(feature_matrix.shape[0])

    Args:
        feature_matrix -  A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        T - An integer indicating how many times the perceptron algorithm
            should iterate through the feature matrix.

    Returns: A tuple where the first element is a numpy array with the value of
    theta, the linear classification parameter, after T iterations through the
    feature matrix and the second element is a real number with the value of
    theta_0, the offset classification parameter, after T iterations through
    the feature matrix.
    """
    theta = np.zeros(feature_matrix.shape[1])
    theta_0 = 0.

    for t in range(T):
        for i in get_order(feature_matrix.shape[0]):
            theta, theta_0 = perceptron_single_step_update(feature_matrix[i, :], labels[i], theta, theta_0)
            pass
    return (theta, theta_0)

In [11]:
feature_matrix = np.array([[1, 0], [1,1], [1, 1.5]])
labels = np.array([1., -1., -1.])
theta = np.zeros(feature_matrix.shape[1])
theta_0 = 0.

In [12]:
def average_perceptron(feature_matrix, labels, T):
    """
    Runs the average perceptron algorithm on a given set of data. Runs T
    iterations through the data set, there is no need to worry about
    stopping early.

    NOTE: Please use the previously implemented functions when applicable.
    Do not copy paste code from previous parts.

    NOTE: Iterate the data matrix by the orders returned by get_order(feature_matrix.shape[0])


    Args:
        feature_matrix -  A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        T - An integer indicating how many times the perceptron algorithm
            should iterate through the feature matrix.

    Returns: A tuple where the first element is a numpy array with the value of
    the average theta, the linear classification parameter, found after T
    iterations through the feature matrix and the second element is a real
    number with the value of the average theta_0, the offset classification
    parameter, found after T iterations through the feature matrix.

    Hint: It is difficult to keep a running average; however, it is simple to
    find a sum and divide.
    """
    dim = feature_matrix.shape[1]
    nrows = feature_matrix.shape[0]
    
    theta = np.zeros(dim)
    theta_sum = np.zeros(dim)
    theta_0 = 0.
    theta_0_sum = 0.

    for t in range(T):
        for i in get_order(nrows):
            theta, theta_0 = perceptron_single_step_update(feature_matrix[i, :], labels [i], theta, theta_0)
            theta_sum += theta
            theta_0_sum += theta_0

    theta_0_sum = theta_0_sum / (T * nrows)
    theta_sum = theta_sum / (T * nrows)
    
    return (theta_sum, theta_0_sum)

In [180]:
average_perceptron(feature_matrix, labels, 5)

In [62]:
def pegasos_single_step_update(
        feature_vector,
        label,
        L,
        eta,
        current_theta,
        current_theta_0):
    """
    Properly updates the classification parameter, theta and theta_0, on a
    single step of the Pegasos algorithm

    Args:
        feature_vector - A numpy array describing a single data point.
        label - The correct classification of the feature vector.
        L - The lamba value being used to update the parameters.
        eta - Learning rate to update parameters.
        current_theta - The current theta being used by the Pegasos
            algorithm before this update.
        current_theta_0 - The current theta_0 being used by the
            Pegasos algorithm before this update.

    Returns: A tuple where the first element is a numpy array with the value of
    theta after the current update has completed and the second element is a
    real valued number with the value of theta_0 after the current updated has
    completed.
    """

    mitad = np.dot(feature_vector, current_theta) + current_theta_0

    if (label * mitad) <= 1:
        current_theta = (1 - eta*L) * current_theta + label * eta * feature_vector
        current_theta_0 += (label * eta)
    else:
        current_theta = (1 - eta*L) * current_theta

    return (current_theta, current_theta_0)

In [63]:
feature_vector = np.array([-0.2044347, 0.32582899, 0.16653248, 0.20351482, -0.25742085, -0.15420557, -0.35653458, -0.11924025, -0.06888927, -0.40490458])

current_theta = np.array([-0.24304381, 0.39144603, -0.16014528, -0.28811461, 0.05726083, -0.37348628, 0.30518816, -0.45640571, 0.08291773, 0.23137106])


label = 1.
L = 0.5184870555601945
eta = 0.5583661127186766

current_theta_0 = 1.019003939895446

In [64]:
pegasos_single_step_update(feature_vector, label, L, eta,
        current_theta, current_theta_0)

(array([-0.28683067,  0.46005208, -0.02079623, -0.09106804, -0.10305158,
        -0.35146307,  0.01775765, -0.39085342,  0.02044715, -0.06169715]),
 1.5773700526141226)

In [108]:
def pegasos(feature_matrix, labels, T, L):
    """
    Runs the Pegasos algorithm on a given set of data. Runs T
    iterations through the data set, there is no need to worry about
    stopping early.

    For each update, set learning rate = 1/sqrt(t),
    where t is a counter for the number of updates performed so far (between 1
    and nT inclusive).

    NOTE: Please use the previously implemented functions when applicable.
    Do not copy paste code from previous parts.

    Args:
        feature_matrix - A numpy matrix describing the given data. Each row
            represents a single data point.
        labels - A numpy array where the kth element of the array is the
            correct classification of the kth row of the feature matrix.
        T - An integer indicating how many times the algorithm
            should iterate through the feature matrix.
        L - The lamba value being used to update the Pegasos
            algorithm parameters.

    Returns: A tuple where the first element is a numpy array with the value of
    the theta, the linear classification parameter, found after T
    iterations through the feature matrix and the second element is a real
    number with the value of the theta_0, the offset classification
    parameter, found after T iterations through the feature matrix.
    """
    
    dim = feature_matrix.shape[1]
    nrows = feature_matrix.shape[0]
    
    theta = np.zeros(dim)
    theta_0 = 0.
    
    t = 1
    for j in range(T):
        for i in get_order(nrows):
            eta = 1 / np.sqrt(t)

            theta, theta_0 = pegasos_single_step_update(feature_matrix[i, :], labels[i], L, eta, theta, theta_0)

            t += 1
    
    return (theta, theta_0)

In [106]:
feature_matrix = np.array([[0.1837462,0.29989789,-0.35889786,-0.30780561,-0.44230703,-0.03043835
,0.21370063,0.33344998,-0.40850817,-0.13105809]
,[0.08254096,0.06012654,0.19821234,0.40958367,0.07155838,-0.49830717
,0.09098162,0.19062183,-0.27312663,0.39060785]
,[-0.20112519,-0.00593087,0.05738862,0.16811148,-0.10466314,-0.21348009
,0.45806193,-0.27659307,0.2901038,-0.29736505]
,[-0.14703536,-0.45573697,-0.47563745,-0.08546162,-0.08562345,0.07636098
,-0.42087389,-0.16322197,-0.02759763,0.0297091,] 
,[-0.18082261,0.28644149,-0.47549449,-0.3049562,0.13967768,0.34904474
,0.20627692,0.28407868,0.21849356,-0.01642202]])


labels = np.array([-1., -1., -1., 1., -1.])
T = 10
L = 0.1456692551041303

In [109]:
pegasos(feature_matrix, labels, T, L)

(array([-0.0788906 , -0.75570849, -0.31067529, -0.05412326, -0.13880071,
         0.09435494, -0.74886923, -0.41457537, -0.07151218, -0.02349776]),
 -0.7607480018258824)

## Ejercicios 7 en adelante

In [3]:
train_data = utils.load_data('reviews_train.tsv')
val_data = utils.load_data('reviews_val.tsv')
test_data = utils.load_data('reviews_test.tsv')

train_texts, train_labels = zip(*((sample['text'], sample['sentiment'])
                                  for sample in train_data))
val_texts, val_labels = zip(*((sample['text'], sample['sentiment'])
                              for sample in val_data))
test_texts, test_labels = zip(*((sample['text'], sample['sentiment'])
                                for sample in test_data))

dictionary = p1.bag_of_words(train_texts)

train_bow_features = p1.extract_bow_feature_vectors(train_texts, dictionary)
val_bow_features = p1.extract_bow_feature_vectors(val_texts, dictionary)
test_bow_features = p1.extract_bow_feature_vectors(test_texts, dictionary)

In [47]:
def classify(feature_matrix, theta, theta_0):
    """
    A classification function that uses theta and theta_0 to classify a set of
    data points.

    Args:
        feature_matrix - A numpy matrix describing the given data. Each row
            represents a single data point.
        theta - A numpy array describing the linear classifier.
        theta_0 - A real valued number representing the offset parameter.

    Returns: A numpy array of 1s and -1s where the kth element of the array is
    the predicted classification of the kth row of the feature matrix using the
    given theta and theta_0. If a prediction is GREATER THAN zero, it should
    be considered a positive classification.
    """
    def less_than_zero(x):
        epsilon = 0.00001
        return x < epsilon

    rtdo = np.array([])
    for i in range(0, feature_matrix.shape[0]):
        mitad = np.dot(feature_matrix[i, :], theta) + theta_0
        if less_than_zero(mitad):
            rtdo = np.append(rtdo, -1.)
        else:
            rtdo = np.append(rtdo, 1.)
            
    return rtdo

In [45]:
feature_matrix = np.array([[1, 0], [1,1], [1, 1.5]])
labels = np.array([1., -1., -1.])
theta = np.zeros(feature_matrix.shape[1])
theta_0 = 0.

In [48]:
classify(feature_matrix, theta, theta_0)

array([1., 1., 1.])

In [53]:
def classifier_accuracy(
        classifier,
        train_feature_matrix,
        val_feature_matrix,
        train_labels,
        val_labels,
        **kwargs):
    """
    Trains a linear classifier and computes accuracy.
    The classifier is trained on the train data. The classifier's
    accuracy on the train and validation data is then returned.

    Args:
        classifier - A classifier function that takes arguments
            (feature matrix, labels, **kwargs) and returns (theta, theta_0)
        train_feature_matrix - A numpy matrix describing the training
            data. Each row represents a single data point.
        val_feature_matrix - A numpy matrix describing the validation
            data. Each row represents a single data point.
        train_labels - A numpy array where the kth element of the array
            is the correct classification of the kth row of the training
            feature matrix.
        val_labels - A numpy array where the kth element of the array
            is the correct classification of the kth row of the validation
            feature matrix.
        **kwargs - Additional named arguments to pass to the classifier
            (e.g. T or L)

    Returns: A tuple in which the first element is the (scalar) accuracy of the
    trained classifier on the training data and the second element is the
    accuracy of the trained classifier on the validation data.
    """
    
    theta, theta_0 = classifier(train_feature_matrix, train_labels, **kwargs)
    
    rtdo_train_labels = classify(train_feature_matrix, theta, theta_0)
    rtdo_val_labels = classify(val_feature_matrix, theta, theta_0)

    train_acc = accuracy(rtdo_train_labels, train_labels)
    val_acc = accuracy(rtdo_val_labels, val_labels)

    return (train_acc, val_acc)

    raise NotImplementedError


In [51]:
train_acc = .5
train_val = .2
type((train_acc, train_val))

tuple

### otras

In [54]:
def extract_words(input_string):
    """
    Helper function for bag_of_words()
    Inputs a text string
    Returns a list of lowercase words in the string.
    Punctuation and digits are separated out into their own words.
    """
    for c in punctuation + digits:
        input_string = input_string.replace(c, ' ' + c + ' ')

    return input_string.lower().split()

In [93]:
def bag_of_words(texts):
    """
    Inputs a list of string reviews
    Returns a dictionary of unique unigrams occurring over the input

    Feel free to change this code as guided by Problem 9
    """
    f = open("stopwords.txt", "r")
    stopwrd = []
    for line in f:
        stopwrd.append(line[:-1])

    dictionary = {}  # maps word to unique index
    for text in texts:
        word_list = extract_words(text)
        for word in word_list:
            if word not in dictionary and word not in stopwrd:
                dictionary[word] = len(dictionary)
    return dictionary

In [ ]:
def extract_bow_feature_vectors(reviews, dictionary):
    """
    Inputs a list of string reviews
    Inputs the dictionary of words as given by bag_of_words
    Returns the bag-of-words feature matrix representation of the data.
    The returned matrix is of shape (n, m), where n is the number of reviews
    and m the total number of entries in the dictionary.

    Feel free to change this code as guided by Problem 9
    """
    # Your code here

    num_reviews = len(reviews)
    feature_matrix = np.zeros([num_reviews, len(dictionary)])

    for i, text in enumerate(reviews):
        word_list = extract_words(text)
        for word in word_list:
            if word in dictionary:
                feature_matrix[i, dictionary[word]] = 1
    return feature_matrix

In [94]:

dictionary = bag_of_words(train_texts)

